<a href="https://colab.research.google.com/github/KunalVaidya99/HappyMonk-Internship-Test/blob/main/HappyMonk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [35]:
banknote = pd.read_csv("/content/BankNote_Authentication.csv")
banknote.head()
x, y = banknote.iloc[:,0:4], banknote.iloc[:,4]
print(y.unique())

[0 1]


In [36]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [98]:
l = x_train.shape[1]
n = 2
p = 2
layer_dims = [l, n, p, 2]

def initialize_parameters(layer_dims):

  np.random.seed(1)

  parameters = dict()

  length = len(layer_dims)

  for i in range(1, length):
    parameters["W" + str(i)] = np.random.randn(layer_dims[i], layer_dims[i-1])
    parameters["b" + str(i)] = np.random.randn(layer_dims[i])

  k0 = np.random.randn(1)
  k1 = np.random.randn(1)
  k2 = np.random.randn(1)

  K = [k0, k1, k2]
  parameters["K"] = K

 
  return parameters                

In [147]:
parameters = initialize_parameters(layer_dims)
parameters

{'K': [array([0.90159072]), array([0.50249434]), array([0.90085595])],
 'W1': array([[ 1.62434536, -0.61175641, -0.52817175, -1.07296862],
        [ 0.86540763, -2.3015387 ,  1.74481176, -0.7612069 ]]),
 'W2': array([[ 1.46210794, -2.06014071],
        [-0.3224172 , -0.38405435]]),
 'W3': array([[-0.17242821, -0.87785842],
        [ 0.04221375,  0.58281521]]),
 'b1': array([ 0.3190391 , -0.24937038]),
 'b2': array([ 1.13376944, -1.09989127]),
 'b3': array([-1.10061918,  1.14472371])}

In [100]:
def softmax(Z):

  activated_output = np.exp(Z)/np.sum(np.exp(Z))
  return activated_output

In [124]:
def forward_propagation(parameters, batch):

  A = batch

  activations = dict()
  layer_outputs = dict()

  activations["A" + str(0)] = batch

  for i in range(1, len(layer_dims)):
   
    Z = np.dot(parameters["W"+str(i)], A) + parameters["b" + str(i)]
    if (i < 3):
      A = parameters["K"][0].item() + parameters["K"][1].item()*Z + parameters["K"][2].item()*(np.multiply(Z,Z))
    else:
      A = softmax(Z)
    activations["A" + str(i)] = A
    layer_outputs["Z"+ str(i)] = Z


  return activations, layer_outputs

In [125]:
activations, layer_outputs = forward_propagation(parameters, np.array([0,1,2,3]))

In [130]:
activations

{'A0': array([0, 1, 2, 3]),
 'A1': array([17.40376153,  1.85522669]),
 'A2': array([478.91123276,  46.81821594]),
 'A3': array([4.8092818e-76, 1.0000000e+00])}

In [131]:
layer_outputs

{'Z1': array([-4.56796669, -1.34490625]),
 'Z2': array([22.75791927, -7.42367129]),
 'Z3': array([-124.77818959,   48.64772972])}

In [103]:
def cross_entropy(predictions, actual):
  loss = -np.mean(np.log(predictions)*actual)
  return loss

In [104]:
def compute_cost(predictions, actual):
  loss = cross_entropy(preecitions, actual)
  return loss

In [136]:
def backward_propagation(parameters, activations, layer_outputs, batch_size, Y):

  gradients = dict()
  gradients["dK"] = 0
  

  for i in reversed(range(1, len(layer_dims))):
    print(i)
    if (i==len(layer_dims)-1):
      gradients["dZ" + str(i)] =  activations["A" + str(i)] - Y
      gradients["dW" + str(i)] = np.dot(activations["A" + str(i-1)].T, gradients["dZ" + str(i)])
      print(gradients["dZ" + str(i)])
      gradients["db" + str(i)] = np.mean(gradients["dZ" + str(i)], axis=0)

      
    if (0 <i < len(layer_dims)-1 ):
      gradients["dA" + str(i)] = np.dot(gradients["dZ" + str(i+1)], parameters["W" + str(i+1)])
      gradients["dZ" + str(i)] = parameters["K"][1].item() + 2*parameters["K"][2].item()*layer_outputs["Z" + str(i)] * activations["A" + str(i)]
      gradients["dW" + str(i)] = np.dot(np.expand_dims(activations["A" + str(i-1)], axis=-1), np.expand_dims(gradients["dZ" + str(i)], axis=-1).T)/batch_size
      gradients["db" + str(i)] = np.mean(gradients["dZ" + str(i)], axis=0)
      gradients["dK" + str(i)] = np.array([np.mean(gradients["dA" +  str(i)]), 
                                           np.mean(np.multiply(gradients["dA" +  str(i)],layer_outputs["Z"+str(i)])), 
                                           np.mean(np.multiply(gradients["dA" + str(i)], layer_outputs["Z"+str(i)]*layer_outputs["Z"+str(i)]))])
      
  for j in range(1, len(layer_dims)-1):
    gradients["dK"] = gradients["dK"] + gradients["dK" + str(i)]     

  return gradients


In [139]:
gradients = backward_propagation(parameters, activations, layer_outputs, 1, [0])
gradients

3
[4.8092818e-76 1.0000000e+00]
2
1


{'dA1': array([ 28913.74003702, -40215.50665761]),
 'dA2': array([0.04221375, 0.58281521]),
 'dK': array([-11301.76662058, -77990.91521723, 530582.62734159]),
 'dK1': array([ -5650.88331029, -38995.45760861, 265291.3136708 ]),
 'dK2': array([ 0.31251448, -1.68296576, 26.99146699]),
 'dW1': array([[   0.        ,    0.        ],
        [-142.73324653,   -3.99296777],
        [-285.46649307,   -7.98593555],
        [-428.1997396 ,  -11.97890332]]),
 'dW2': array([[341764.66551987, -10889.63766955],
        [ 36431.83281963,  -1160.82643722]]),
 'dW3': 46.81821593828207,
 'dZ1': array([-142.73324653,   -3.99296777]),
 'dZ2': array([19637.40223062,  -625.70598028]),
 'dZ3': array([4.8092818e-76, 1.0000000e+00]),
 'db1': -73.36310715460812,
 'db2': 9505.848125170538,
 'db3': 0.5}

In [144]:
def update_parameters(parameters, gradients, learning_rate):

  for i in range(1, len(layer_dims)):
    parameters["W" + str(i)] = parameters["W" + str(i)] - learning_rate * gradients["dW" + str(i)].T
    parameters["b" + str(i)] = parameters["b" + str(i)] - learning_rate * gradients["db" + str(i)]

  parameters["K"] = parameters["K"] - learning_rate * gradients["dK"] 

  return parameters 

In [146]:
updated_parameters = update_parameters(parameters, gradients, 0.001)
updated_parameters

{'K': array([[   23.50512396,   156.88342116, -1060.26366396],
        [   23.10602758,   156.48432477, -1060.66276034],
        [   23.50438919,   156.88268638, -1060.26439873]]),
 'W1': array([[ 1.62434536, -0.18355667,  0.32822773,  0.2116306 ],
        [ 0.86540763, -2.28955979,  1.76876957, -0.72527019]]),
 'W2': array([[-682.0672231 ,  -74.92380635],
        [  21.45685814,    1.93759852]]),
 'W3': array([[-0.26606464, -0.97149485],
        [-0.05142269,  0.48917878]]),
 'b1': array([ 0.46576531, -0.10264416]),
 'b2': array([-17.87792681, -20.11158752]),
 'b3': array([-1.10161918,  1.14372371])}